# Setting things up

## About this notebook

In this notebook, we embark on a journey to enhance the predictive power of the Titanic Dataset through sophisticated preprocessing using the ``BinaryCarver`` pipeline. Designed to maximize associations in the data, ``BinaryCarver`` is a robust Python tool capable of discretizing any type of data—whether it be quantitative or qualitative. Our specific focus is on preparing the dataset for binary classification tasks, such as predicting survival outcomes.

The Titanic Dataset, derived from the iconic 1912 Titanic passenger information, provides a diverse set of features ranging from socio-economic status and age to cabin location. Leveraging ``BinaryCarver``, we aim to perform association-maximizing discretization, refining both quantitative and qualitative features to create a finely tuned dataset for our binary classification endeavors.

Throughout this notebook, we'll delve into the intricacies of ``BinaryCarver``'s discretization pipeline, exploring its capabilities in handling a variety of data types. Whether it's transforming passenger ages or classifying fares, ``BinaryCarver``'s adaptability ensures that every feature is optimally represented for our classification tasks.

Join us in this exploration as we harness the power of ``BinaryCarver`` to preprocess the Titanic Dataset. Through effective feature engineering and discretization, we strive to create a dataset that not only captures the nuances of the Titanic passenger profiles but also sets the stage for the development of accurate and impactful binary classification models.

Let's dive in and uncover the potential of ``BinaryCarver`` in transforming the Titanic Dataset for optimal predictive modeling.


## Installation

In [1]:
# %pip install AutoCarver[jupyter]

## Titanic Data

In this example notebook, we will use the Titanic dataset.

The Titanic dataset is a well-known and frequently used dataset in the field of machine learning and data science. It provides information about the passengers on board the Titanic, the famous ship that sank on its maiden voyage in 1912. The dataset is often used for predictive modeling, classification, and regression tasks.

The dataset includes various features such as passengers' names, ages, genders, ticket classes, cabin information, and whether they survived or not. The primary goal when working with the Titanic dataset is often to build predictive models that can infer whether a passenger survived or perished based on their individual characteristics (binary classification).

In [2]:
import pandas as pd

# URL to the Titanic dataset on Kaggle
titanic_url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"

# Use pandas to read the CSV file directly from the URL
titanic_data = pd.read_csv(titanic_url)

# Display the first few rows of the dataset
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


## Target type and Carver selection

In [3]:
target = "Survived"

titanic_data[target].value_counts(dropna=False)

Survived
0    545
1    342
Name: count, dtype: int64

The target ``"Survived"`` is a binary target of type ``int64`` used in a classification task. Hence we will use ``AutoCarver.BinaryCarver`` and ``AutoCarver.selectors.ClassificationSelector`` in following code blocks.

## Data Sampling

In [4]:
from sklearn.model_selection import train_test_split

# stratified sampling by target
train_set, dev_set = train_test_split(titanic_data, test_size=0.33, random_state=42, stratify=titanic_data[target])

# checking target rate per dataset
train_set[target].mean(), dev_set[target].mean()

(np.float64(0.38552188552188554), np.float64(0.3856655290102389))

## Setting up Features to Carver

In [5]:
train_set.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
617,0,3,Mr. Antoni Yasbeck,male,27.0,1,0,14.4542
489,0,1,Mr. Harry Markland Molson,male,55.0,0,0,30.5000
871,1,3,Miss. Adele Kiamie Najib,female,15.0,0,0,7.2250
654,0,3,Mrs. John (Catherine) Bourke,female,32.0,1,1,15.5000
653,0,3,Mr. Alexander Radeff,male,27.0,0,0,7.8958


In [6]:
# column data types
train_set.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [7]:
# values taken by Parents/Children Aboard
train_set["Parents/Children Aboard"].value_counts()

Parents/Children Aboard
0    438
1     87
2     60
3      3
5      3
4      2
6      1
Name: count, dtype: int64

In [8]:
# values taken by Pclass
train_set["Pclass"].value_counts()

Pclass
3    326
1    142
2    126
Name: count, dtype: int64

The feature ``"Pclass"`` is of type ``"int64"``, but it can be considered a qualitative ordinal feature rather than a quantitative discrete feature (socio-economic status). Thus we will add it to the list of ``ordinal_features`` and set the ordering of its values in ``values_orders`` (string values). 

``"Sex"`` is the only quantitative categorical feature, it's added to the list of ``qualitative_features``.

``"Fare"`` is the only quantitative continuous features, whilst ``"Age"``, ``"Siblings/Spouses Aboard"`` and ``"Parents/Children Aboard"`` can be considered as quantitative discrete features. Those four features will be added to the list of ``quantitative_features``.

In [9]:
from AutoCarver import Features

# initiating Features to carve
features = Features(
    categoricals=["Sex"],
    quantitatives=["Age", "Fare", "Siblings/Spouses Aboard", "Parents/Children Aboard"],
    ordinals={"Pclass": ["1", "2", "3"]},  # user-specified ordering for ordinal features
)
features["Pclass"], features["Sex"], features["Age"]

C:\Users\defra\Desktop\git\PROJECTS\AutoCarver\AutoCarver\combinations\utils\combination_evaluator.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


(Ordinal('Pclass'), Categorical('Sex'), Quantitative('Age'))

# Using AutoCarver

## AutoCarver settings

### Representativness of modalities

The attribute ``min_freq`` allows one to choose the minimum frequency per basic modalities. It is used:

- For quantitative features, to define the number of quantiles to initialy discretize the features with.

- For qualitative features, to define the threshold under which a modality is grouped to either a default value or its closest modality.

In [10]:
min_freq = 0.05

**Tip:** should be set between ``0.01`` (slower, preciser, less robust) and ``0.2`` (faster, more robust)

### Optional: Desired number of modalities

The attribute ``max_n_mod`` allows one to choose the maximum number of modalities per carved feature. It is used by **Carvers** has the upper limit of number of modalities per consecutive combination of modalities.

In [11]:
max_n_mod = 5

**Tip:** should be set between ``3`` (faster, more robust) and ``7`` (slower, preciser, less robust)

### Optional: Grouping NaNs

The attribute ``dropna`` allows one to choose whether or not ``nan`` should be grouped with another modality. If set to ``True``, **Carvers** will first find the most suitable combination of non-``nan`` values, and then test out all possible combinations with ``nan``.

In [12]:
dropna = False  # anyway, there are no nan in this dataset


## Fitting AutoCarver

* First, all qualitative features are discretized:
    1. Using ``StringDiscretizer`` to convert them to ``str`` if not already the case
    2. For qualitative ordinal features: using ``OrdinalDiscretizer`` for under-represented values (less frequent than ``min_freq``) to be grouped with its closest modality
    3. For qualitative categorical features: using ``CategoricalDiscretizer`` for under-represented values (less frequent than ``min_freq``) to be grouped with a default value (``features.default="__OTHER__"``)

* Second, all quantitative features are discretized:
    1. Using ``ContinuousDiscretizer`` for quantile discretization that keeps track of over-represented values (more frequent than ``min_freq``)
    2. Using ``OrdinalDiscretizer`` for any remaining under-represented values (less frequent than ``min_freq/2``) to be grouped with its closest modality

* Third, all features are carved following this recipe, for all classes of ``train_set[target]`` (except one):
    1. The raw distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the discretization step
    2. Grouping modalities: all consecutive combinations of modalities are applied to ``train_set``
    3. Computing associations: the association metric (Tschruprow's T, by default) is computed with the provided ``train_set[target]``
    4. Combinations are sorted in descending order by association value
    5. Testing robustness: finds the first combination that checks the following:
        - Representativness of modalities on ``train_set`` and ``dev_set`` (all should be more frequent than ``min_freq/2``)
        - Distinct target rates per consecutive modalities on ``train_set`` and ``dev_set`` 
        - No inversion of target rates between ``train_set`` and ``dev_set`` (same ordering of modalities by target rate)
    6. (Optional) If requested via ``dropna=True``, and if any, all combinations of modalities with ``nan`` are applied to ``train_set`` and steps 3. and 4. are run
    7. The carved distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the carving step

In [13]:
from AutoCarver import BinaryCarver

# intiating AutoCarver
auto_carver = BinaryCarver(
    features=features,
    min_freq=min_freq,
    dropna=dropna,
    verbose=True,  # showing statistics
    copy=True,  # whether or not to return a copy of the input dataset
)

# fitting on training sample, a dev sample can be specified to evaluate carving robustness
train_set_processed = auto_carver.fit_transform(train_set, train_set[target], X_dev=dev_set, y_dev=dev_set[target])

------
--- [QuantitativeDiscretizer] Fit Features(['Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
 - [ContinuousDiscretizer] Fit Features(['Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
 - [OrdinalDiscretizer] Fit Features(['Age', 'Fare', 'Parents/Children Aboard'])
------

------
--- [QualitativeDiscretizer] Fit Features(['Sex', 'Pclass'])
 - [StringDiscretizer] Fit Features(['Pclass'])
 - [OrdinalDiscretizer] Fit Features(['Pclass'])
 - [CategoricalDiscretizer] Fit Features(['Sex'])
------

---------
------ [BinaryCarver] Fit Features(['Sex', 'Pclass', 'Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
--- [BinaryCarver] Fit Categorical('Sex') (1/6)
 [BinaryCarver] Raw distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 male 
 0.1878 
 0.6364 
 
 
 female 
 0.7315 
 0.3636 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.1949 
 0.6655 
 
 
 0.7653 
 0.3345

Testing robustness    :   0%|          | 0/1 [00:00<?, ?it/s]



 [BinaryCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 male 
 0.1878 
 0.6364 
 
 
 female 
 0.7315 
 0.3636 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.1949 
 0.6655 
 
 
 0.7653 
 0.3345

...

X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.0e+00 
 0.3447 
 0.7374 
 
 
 0.0e+00 < x <= 1.0e+00 
 0.5057 
 0.1465 
 
 
 1.0e+00 < x <= 2.0e+00 
 0.5167 
 0.1010 
 
 
 2.0e+00 < x 
 0.3333 
 0.0152 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.6774 
 0.1058 
 
 
 0.4500 
 0.0683 
 
 
 0.1667 
 0.0205

Testing robustness    :   0%|          | 0/7 [00:00<?, ?it/s]



 [BinaryCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.0e+00 
 0.3447 
 0.7374 
 
 
 0.0e+00 < x 
 0.5000 
 0.2626 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.5439 
 0.1945

C:\Users\defra\Desktop\git\PROJECTS\AutoCarver\AutoCarver\discretizers\utils\base_discretizer.py:413: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sample.X.replace(


## AutoCarver analysis

### Carving Summary

In [14]:
auto_carver.summary

content  \
feature                                 cramerv  tschuprowt n_mod label                           
Categorical('Sex')                      0.533719 0.533719   2     0                        male   
                                                                  1                      female   
Ordinal('Pclass')                       0.300144 0.300144   2     0                      [2, 1]   
                                                                  1                           3   
Quantitative('Age')                     0.139166 0.139166   2     0                x <= 8.0e+00   
                                                                  1                 8.0e+00 < x   
Quantitative('Fare')                    0.295325 0.295325   2     0                x <= 5.2e+01   
                                                                  1                 5.2e+01 < x   
Quantitative('Siblings/Spouses Aboard') 0.139722 0.117492   3     0                x <= 0.0e+00   
                                                                  1      0.0e+00 < x <= 1.0e+00   
                                                                  2                 1.0e+00 < x   
Quantitative('Parents/Children Aboard') 0.136439 0.136439   2     0                x <= 0.0e+00   
                                                                  1                 0.0e+00 < x   

                                                                         target_rate  \
feature                                 cramerv  tschuprowt n_mod label                
Categorical('Sex')                      0.533719 0.533719   2     0         0.187831   
                                                                  1         0.731481   
Ordinal('Pclass')                       0.300144 0.300144   2     0         0.548507   
                                                                  1         0.251534   
Quantitative('Age')                     0.139166 0.139166   2     0         0.636364   
                                                                  1         0.365455   
Quantitative('Fare')                    0.295325 0.295325   2     0         0.319838   
                                                                  1         0.710000   
Quantitative('Siblings/Spouses Aboard') 0.139722 0.117492   3     0         0.361386   
                                                                  1         0.500000   
                                                                  2         0.269231   
Quantitative('Parents/Children Aboard') 0.136439 0.136439   2     0         0.344749   
                                                                  1         0.500000   

                                                                         frequency  
feature                                 cramerv  tschuprowt n_mod label             
Categorical('Sex')                      0.533719 0.533719   2     0       0.636364  
                                                                  1       0.363636  
Ordinal('Pclass')                       0.300144 0.300144   2     0       0.451178  
                                                                  1       0.548822  
Quantitative('Age')                     0.139166 0.139166   2     0       0.074074  
                                                                  1       0.925926  
Quantitative('Fare')                    0.295325 0.295325   2     0       0.831650  
                                                                  1       0.168350  
Quantitative('Siblings/Spouses Aboard') 0.139722 0.117492   3     0       0.680135  
                                                                  1       0.232323  
                                                                  2       0.087542  
Quantitative('Parents/Children Aboard') 0.136439 0.136439   2     0       0.737374  
                                                                  1       0.262626

* For quantitative feature ``Age``, the selected combination of modalities groups ages as follows:
    * modality ``0``: lower or equal to 8 years old (``content="x <= 8.0+00"``)
    * modality ``1``: ages higher than 8 years old (``content="8.0+00 < x "``)

* For qualitative categorical feature ``Sex``, the selected combination of modalities has left modalities ``content="male"`` in modality ``0`` and ``content="female"`` in modality ``1`` (no combination possible)

* For qualitative ordinal feature ``Pclass``, the selected combination of modalities socio-economic status as follows:
    * modality ``0``: upper and middle classes (``content=[2, 1]``) 
    * modality ``1``: lower class (``content=3``). 
    * The user-provided ordering of modalities has been preserved.

### Detailed overview of tested combinations

In [15]:
features["Pclass"].history

,info,cramerv,tschuprowt,combination,n_mod,dropna,train,viable,dev
0,Raw distribution,0.321044,0.269965,"{'1': '1', '2': '2', '3': '3'}",3,False,NaN,NaN,NaN
1,Best for tschuprowt and max_n_mod=5,0.300144,0.300144,"{'1': '1', '2': '1', '3': '3'}",2,False,"{'viable': True, 'info': ''}",True,"{'viable': True, 'info': ''}"
2,Not checked,0.321044,0.269965,"{'1': '1', '2': '2', '3': '3'}",3,False,NaN,NaN,NaN
3,Not checked,0.265643,0.265643,"{'1': '1', '2': '2', '3': '2'}",2,False,NaN,NaN,NaN


* The most associated combination (the first tested out, where ``info!="Raw distribution"``) groups ``Pclass==1`` with ``Pclass==2`` and leaves ``Pclass==3`` as its own modality

* For feature ``Pclass``, the 1st combination passes the tests:
    - ``viable=True``
    - ``info="Best for tschuprowt and max_n_mod=5"``
    - Tschuprow's T with ``Survived`` is ``0.300144`` for this combination (by default, combinations are ranked according to this statistic)
    - Following combinations (less associated with the target) where not tested: ``info="Not checked"``

* For all combinations ``dropna=False`` means that it is not a combination in which ``nan``s are being grouped with other modalities (as requested with ``dropna=False``)

## Saving and Loading AutoCarver
### Saving

All **Carvers** can safely be stored as a .json file.

In [16]:
auto_carver.save("binary_carver.json")

### Loading

**Carvers** can safely be loaded from a .json file.

In [17]:
auto_carver = BinaryCarver.load("binary_carver.json")

## Applying AutoCarver

In [18]:
dev_set_processed = auto_carver.transform(dev_set)

C:\Users\defra\Desktop\git\PROJECTS\AutoCarver\AutoCarver\discretizers\utils\base_discretizer.py:413: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sample.X.replace(


In [19]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

,Sex,Pclass,Age,Fare,Siblings/Spouses Aboard,Parents/Children Aboard
0.0,0.665529,0.450512,0.064846,0.832765,0.682594,0.805461
1.0,0.334471,0.549488,0.935154,0.167235,0.242321,0.194539
2.0,NaN,NaN,NaN,NaN,0.075085,NaN


# Feature Selection
## Selectors settings
### Features to select from

Here all features have been carved using ``BinaryCarver``, hence all features are qualitative.

### Number of features to select

The attribute ``n_best_per_type`` allows one to choose the number of features to be selected per data type (quantitative and qualitative).

In [20]:
n_best_per_type = 4  # here the number of features is low, ClassificationSelector will only be used to compute useful statistics

## Using Selectors

In [21]:
from AutoCarver import ClassificationSelector

# select the most target associated qualitative features
feature_selector = ClassificationSelector(
    features=features,
    n_best_per_type=n_best_per_type,
    verbose=True,  # displays statistics
)
best_features = feature_selector.select(train_set_processed, train_set_processed[target])
best_features

 [ClassificationSelector] Selected Features


,feature,NanMeasure,ModeMeasure,TschuprowtMeasure,TschuprowtRank,TschuprowtFilter,TschuprowtWith
0,Categorical('Sex'),0.0000,0.6364,0.5337,0,0.0000,itself
1,Ordinal('Pclass'),0.0000,0.5488,0.3001,1,0.0988,Sex
3,Quantitative('Fare'),0.0000,0.8316,0.2953,2,0.3922,Pclass
2,Quantitative('Age'),0.0000,0.9259,0.1392,3,0.1002,Sex
5,Quantitative('Parents/Children Aboard'),0.0000,0.7374,0.1364,4,0.4666,Age
4,Quantitative('Siblings/Spouses Aboard'),0.0000,0.6801,0.1175,5,0.4060,Parents/Children Aboard


Features(['Sex', 'Pclass', 'Fare', 'Age'])

In [22]:
train_set_processed[best_features].head()

,Sex,Pclass,Fare,Age
617,0,1,0.0,1.0
489,0,0,0.0,1.0
871,1,1,0.0,1.0
654,1,1,0.0,1.0
653,0,1,0.0,1.0


* Feature ``Sex`` is the most associated with the target ``Survived``:
    - Tschuprow's T value is ``TschuprowtMeasure=0.5337``
    - It has 0 % of NaNs (``NaNMeasure=0.0``) 
    - Its mode represents 64 % of observed data (``ModeMeasure=0.6364``)

* Feature ``Fare`` is strongly associated to feature ``Pclass``:
    - Tschuprow's T value is ``TschuprowtFilter=0.3922`` with ``TschuprowtWith=Pclass``

* Here, no feature where filtered out for there inter-feature association or over-represented values (no thresholds were set)

# Modeling
Fitting model on train data

In [23]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(train_set_processed[best_features], train_set_processed[target])

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Saving model

In [24]:
model.save_model("binary_xgboost.json")

Prediction on dev dataset and performance

In [25]:
from sklearn.metrics import roc_auc_score

dev_pred = model.predict_proba(dev_set_processed[best_features])[:, 1]
roc_auc_score(dev_set_processed[target], dev_pred)

np.float64(0.8548426745329402)

## What's next?

* Thanks to **Carvers** all of your features are now optimally processed for your classification task!
* As a final step towards your model, **Selectors** can prove to be handy tools to operate target optimal Data Pre-Selection, so make sure to check out [Selectors Examples](https://autocarver.readthedocs.io/en/latest/selectors_examples.html)!

## Well done!

Your commitment to achieving optimal results in binary classification tasks shines through in your meticulous use of **AutoCarver**'s ``BinaryCarver`` for data preprocessing. By fine-tuning and optimizing your dataset, you have set the stage for robust and accurate machine learning models.

The ``BinaryCarver`` has proven to be a valuable ally in your pursuit of excellence, carving out a path toward enhanced feature representation and model interpretability. Your dedication to refining the data preprocessing steps reflects a commitment to extracting the maximum value from your datasets.

We extend our sincere appreciation for choosing **AutoCarver** as your companion in the data preprocessing journey. Your use of **AutoCarver** demonstrates a dedication to leveraging cutting-edge tools for achieving excellence in binary classification tasks.

As you transition to the modeling phase, may the carefully crafted features and preprocessing steps contribute to the success of your predictive models. We're excited to see the impact of your work and are grateful for the opportunity to be part of your data science endeavors.

Thank you for trusting **AutoCarver**, and we wish you continued success in your data-driven ventures.